## Trabalho Parte 1 - Alunas: Ainhoa Acinas Amparo Báguena

### INTRODUÇÃO

Este trabalho consiste em avaliar e classificar a qualidade de automóveis utilizando um conjunto de dados específico (“Car Evaluation Data Set”). Os dados podem ser descarregados a partir da seguinte ligação: https://drive.google.com/file/d/1RfFy8n53YqRDMCrVq5wEDEGGkqWMAPSb/view?usp=drive_link.
O objetivo é construir um modelo que, com base em caraterísticas como o preço de compra, preço de manutenção, número de portas, capacidade de passageiros, tamanho da bagageira e segurança estimada, determine a qualidade de um carro em quatro categorias possíveis: `unacc` (inaceitável), `acc` (aceitável), `good` (bom), e `vgood` (muito bom).
Vários modelos de aprendizagem automática serão implementados e comparados para escolher o modelo mais eficaz para esta tarefa de classificação

### Leitura de dados

Em primeiro lugar, importamos as bibliotecas que vamos utilizar para realizar o nosso projeto.

In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn import tree
from sklearn.svm import SVC
from sklearn import naive_bayes
from scipy.optimize import minimize
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid

Agora vamos carregar os dados do ficheiro com que vamos trabalhar.

In [2]:
nombres_columnas = ["buying", "maint", "doors", "persons", "lug_boot", "safety","class"]
df = pd.read_csv('car.data', delimiter=',', header=None, names=nombres_columnas)
df


,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


Convertemos todas as colunas em valores numéricos para podermos aplicar os modelos. Além disso, separamos os 6 atributos que vão fazer parte do X do que vai fazer parte do y, que será a variável a ser prevista.

In [3]:
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = LabelEncoder().fit_transform(df[column])
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,2,1
1,3,3,0,0,2,2
2,3,3,0,0,2,0
3,3,3,0,0,1,1
4,3,3,0,0,1,2
...,...,...,...,...,...,...
1723,1,1,3,2,1,2
1724,1,1,3,2,1,0
1725,1,1,3,2,0,1
1726,1,1,3,2,0,2


Separamos os dados do nosso ficheiro em dois conjuntos, treino e validação, para podermos escolher mais tarde o nosso melhor modelo.

In [4]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=0.3,random_state=123)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size = 0.3, random_state = 123)

### Análise  do modelo

Em primeiro lugar, vamos criar um dicionário no qual vamos armazenar as melhores combinações de parâmetros que obtemos de cada modelo, bem como as suas precisões na validação.

In [5]:
best_params={}
best_accs={}

#### Regressão logística

Em primeiro lugar, vamos utilizar o modelo de regressão logística. Para o fazer, vamos treinar o modelo com a classe LogisticRegression do scikit-learn e utilizar o método 'newton-cg' para encontrar o mínimo. 

In [6]:
LogReg = LogisticRegression(penalty ='l2', C = 10.0e50, max_iter = 400, solver = 'newton-cg')
LogReg = LogReg.fit(Xtrain, ytrain.ravel())

ypredtrain = LogReg.predict(Xtrain).reshape(-1, 1)
acctrain = accuracy_score(ypredtrain, ytrain)

ypredval = LogReg.predict(Xval).reshape(-1, 1)
accval = accuracy_score(ypredval, yval)

print("Precisão no conjunto de treino: {}".format(acctrain))
print("Precisão no conjunto de validação: {}".format(accval))

best_accs['Regresión Logística'] = accval

Precisão no conjunto de treino: 0.6843971631205674
Precisão no conjunto de validação: 0.6639118457300276


Verificamos que a precisão nos conjuntos de treino e de validação não é muito elevada. Por isso, vamos experimentar a regressão logística com regularização para ver se melhora um pouco a precisão.

#### Regressão logística com regularização

Em segundo lugar, utilizaremos este método com regularização.

Em primeiro lugar, aplicaremos as transformações necessárias aos dados para classificar corretamente os dados correspondentes à malha do plano.

Posteriormente, utilizaremos a biblioteca scikit-learn para treinar o modelo.

In [7]:
poly = PolynomialFeatures(4)
Xpoly_train = poly.fit_transform(Xtrain)
Xpoly_val = poly.transform(Xval)

scaler = StandardScaler()
Xpoly_norm_train = scaler.fit_transform(Xpoly_train)
Xpoly_norm_val = scaler.transform(Xpoly_val)

In [8]:
accstrain = []
accsval = []
valores_C=[np.inf, 1, 1/100] 

for i, valor_lambda in enumerate(valores_C):
    LogRegu = LogisticRegression(penalty = 'l2', C = valor_lambda, max_iter = 500, solver = 'newton-cg')
    LogRegu = LogRegu.fit(Xpoly_norm_train, ytrain.ravel())
   
    ypredtrain = LogRegu.predict(Xpoly_norm_train).reshape(-1, 1)
    ypredval = LogRegu.predict(Xpoly_norm_val).reshape(-1, 1)
    
    accstrain.append(accuracy_score(ypredtrain, ytrain))
    accsval.append(accuracy_score(ypredval, yval))

print("Precisão no conjunto de treino: {}".format(accstrain))
print("Precisão no conjunto de validação: {}".format(accsval))

best_accs['Regresión Logística con Regularización'] = np.max(accsval)
best_params['Regresión Logística con Regularización'] = valores_C[np.argmax(accsval)]

Precisão no conjunto de treino: [1.0, 0.9964539007092199, 0.8132387706855791]
Precisão no conjunto de validação: [0.9421487603305785, 0.9338842975206612, 0.790633608815427]


#### KNN e classificação

Agora vamos aplicar o algoritmo KNN disponível no Scikit-Learn. No KNN, classificamos um novo exemplo tendo em conta os k exemplos mais próximos. Em vez de o utilizarmos com um k selecionado a priori, vamos experimentar com diferentes k e manteremos aquele que nos der o melhor resultado.

Esta implementação encontra-se no pacote neighbours e a sua classe específica é KNeighborsClassifier.

In [9]:
vals_acc_test_ponderado = np.zeros(12)
accmax = 0
kmax = 0
for k in range(12):
    knn_clasif = KNeighborsClassifier(n_neighbors = k+1)
    knn_clasif.fit(Xtrain, ytrain)
    ypredtrain = knn_clasif.predict(Xtrain)
    ypredval = knn_clasif.predict(Xval)
    acctrain = accuracy_score(ypredtrain, ytrain)
    accval = accuracy_score(ypredval, yval)
    print("\nPrecisão no conjunto de treino com k={}: {}".format(k+1, acctrain))
    print("Precisão no conjunto de validação com k={}: {}".format(k+1, accval))
    
    if accval > accmax:
        accmax = accval
        kmax = k+1
    
print('\nO melhor valor para k é  {}, com uma precisão no conjunto de validação de {}'.format(kmax, accmax))

best_accs['KNN'] = accmax
best_params['KNN'] = kmax


Precisão no conjunto de treino com k=1: 1.0
Precisão no conjunto de validação com k=1: 0.7603305785123967

Precisão no conjunto de treino com k=2: 0.8711583924349882
Precisão no conjunto de validação com k=2: 0.7245179063360881

Precisão no conjunto de treino com k=3: 0.9349881796690307
Precisão no conjunto de validação com k=3: 0.8539944903581267

Precisão no conjunto de treino com k=4: 0.9231678486997635
Precisão no conjunto de validação com k=4: 0.8292011019283747

Precisão no conjunto de treino com k=5: 0.9373522458628841
Precisão no conjunto de validação com k=5: 0.8815426997245179

Precisão no conjunto de treino com k=6: 0.9302600472813238
Precisão no conjunto de validação com k=6: 0.859504132231405

Precisão no conjunto de treino com k=7: 0.9243498817966903
Precisão no conjunto de validação com k=7: 0.8677685950413223

Precisão no conjunto de treino com k=8: 0.9160756501182034
Precisão no conjunto de validação com k=8: 0.8650137741046832

Precisão no conjunto de treino com k=9:

Utilizaremos então k = 5 vizinhos. Passamos a comparar diferentes métricas no nosso algoritmo.

Para comparação, utilizaremos as seguintes distâncias:
* Distância de Manhattan
* Distância Euclidiana
* Distância de Minkowski com p = 1,5
* Semelhança de cosseno (disponível com a métrica='cosine'): $$d_c(\mathbf{x}, \mathbf{y}) = \frac{\mathbf{x}\cdot \mathbf{y}}{\Vert \mathbf{x}\Vert \Vert \mathbf{y} \Vert }$$ 


In [10]:
# Debemos guardar el accuracy de test en acc_test
clasif1 = KNeighborsClassifier(n_neighbors = 5, metric = 'manhattan')
clasif2 = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')
clasif3 = KNeighborsClassifier(n_neighbors = 5, p = 1.5)
clasif4 = KNeighborsClassifier(n_neighbors = 5, metric = 'cosine')

clasificadores = [clasif1, clasif2, clasif3, clasif4]
acc = []

for clasificador in clasificadores:
    clasificador.fit(Xtrain, ytrain)
    
    ypredval = clasificador.predict(Xval)
    accval = accuracy_score(ypredval, yval)
    acc.append(accval)
    
    ypredtrain = clasificador.predict(Xtrain)
    acctrain = accuracy_score(ypredtrain, ytrain)
    
    print('\nClassificador {}. \nPrecisão no conjunto de treino: {}. Precisão no conjunto de validação: {}'.format(clasificador, acctrain, accval))

mejorAcc = np.max(acc)
mejorClasif = clasificadores[acc.index(mejorAcc)]

mejorClasif.fit(Xtrain, ytrain)

ypredtrain = mejorClasif.predict(Xtrain)
ypredval = mejorClasif.predict(Xval)

acctrain = accuracy_score(ypredtrain,ytrain)
accval = accuracy_score(ypredval,yval)

print('\nO melhor classificador é: {}, com uma precisão no conjunto de validação de : {}'.format(mejorClasif, accval))

best_accs['KNN distancias'] = accval
best_params['KNN distancias'] = mejorClasif


Classificador KNeighborsClassifier(metric='manhattan'). 
Precisão no conjunto de treino: 0.9397163120567376. Precisão no conjunto de validação: 0.8677685950413223

Classificador KNeighborsClassifier(metric='euclidean'). 
Precisão no conjunto de treino: 0.9373522458628841. Precisão no conjunto de validação: 0.8815426997245179

Classificador KNeighborsClassifier(p=1.5). 
Precisão no conjunto de treino: 0.9373522458628841. Precisão no conjunto de validação: 0.8705234159779615

Classificador KNeighborsClassifier(metric='cosine'). 
Precisão no conjunto de treino: 0.9078014184397163. Precisão no conjunto de validação: 0.8292011019283747

O melhor classificador é: KNeighborsClassifier(metric='euclidean'), com uma precisão no conjunto de validação de : 0.8815426997245179


Como mostra o código, o melhor classificador é o que utiliza a distância euclidiana e que nos dá a melhor precisão no conjunto de validação até agora.

#### Naïve Bayes classificação

De seguida, utilizaremos o algoritmo Naïve Bayes, que prevê a classe da instância de teste com a maior probabilidade.

Decidimos testar com este algoritmo devido à sua rapidez em treinar os dados e classificá-los.

In [11]:
NaiveBayes = naive_bayes.GaussianNB()
NaiveBayes=NaiveBayes.fit(Xtrain,ytrain)

ypredtrain = NaiveBayes.predict(Xtrain)
acctrain = accuracy_score(ypredtrain, ytrain)

ypredval = NaiveBayes.predict(Xval)
accval = accuracy_score(ypredval, yval)

print("Precisão no conjunto de treino: {}".format(acctrain))
print("Precisão no conjunto de validação: {}".format(accval))
best_accs['Naive Bayes']=accval

Precisão no conjunto de treino: 0.5981087470449172
Precisão no conjunto de validação: 0.581267217630854


Observamos que, para este modelo, obtemos uma precisão no conjunto de validação de 58,13%, o que é bastante baixo em comparação com os modelos anteriores. Por conseguinte, podemos concluir que este método não é muito adequado para o nosso conjunto de dados.

#### Redes Neurais

Neste ponto, vamos testar os algoritmos de *Redes Neurais*, que são algoritmos que tentam imitar o cérebro humano. 

Primeiro, como vimos na teoria, para trabalhar com problemas multi-classe com redes neurais precisamos transformar a saída que é um único valor (a classe) em um vetor de 0s e 1s, onde o 1 na posição correspondente nos diz a qual classe o exemplo pertence. Isto é necessário porque vamos ter um neurónio de saída para cada uma das classes. Para conseguir esta transformação, vamos utilizar o método *OneHotEncoder do Scikit-Learn*.

In [12]:
# Instancia un objeto de tipo OneHotEncoder, con sparse=False
y_train_reshaped = np.array(ytrain).reshape(-1, 1)

# Instancia el OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Aplica fit_transform a y_train_reshaped
y_onehot = encoder.fit_transform(y_train_reshaped)
print(y_onehot.shape)

(846, 4)


C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


A primeira coisa que precisamos de fazer é implementar a função de custo para avaliar o custo para um determinado conjunto de parâmetros. Para isso, precisamos primeiro de aplicar a propagação para a frente, que nos dá o resultado para cada exemplo de entrada. Para isso, começaremos por implementar a função sigmoide.

In [13]:
def sigmoid(z):
    z = 1/(1 + np.exp(-z))
    return z

De seguida, calculamos a propagação para a frente, tal como aprendemos na teoria e na prática.

In [14]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    # Añadimos la columna de unos a X para obtener a1
    a1 = np.hstack((np.ones(m).reshape(-1, 1), X))
    
    # Calculamos z2 
    # Por otro lado, theta1 tiene tantas filas como neuronas en la capa oculta y columnas como atributos + 1
    z2 = np.dot(a1, theta1.T)
    
    # Añadimos la columna de unos a la sigmoide de z2 (que es a2) para obtener el a2 definitivo
    a2 = sigmoid(z2)
    unos = np.ones((a2.shape[0], 1))
    a2 = np.hstack((unos, a2))

    # Calculamos z3
    z3 = np.dot(a2, theta2.T)
    
    # Obtenemos la salida final en h
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

Com a propagação para a frente, podemos agora calcular a função de custo. No entanto, antes de a calcular, preparamos outras funções que serão necessárias mais tarde.

In [15]:
# Configuración inicial
input_size = 6
hidden_size = 10
num_labels = 4
np.random.seed(123456789)

# Inicializamos los parámetros de la red aleatoriamente
# El tamaño del array es el tamaño de las dos matrices de pesos concatenadas
params = (np.random.random(size = hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

# Podemos desempaquetar los parámetros que acabamos de inicializar igual que lo hacemos en la función de coste
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

# Vemos los tamaños de las matrices theta1 y theta2
theta1.shape, theta2.shape

((10, 7), (4, 11))

Verificamos se o que foi implementado até agora está a funcionar corretamente. 

In [16]:
a1, z2, a2, z3, h = forward_propagate(Xtrain, theta1, theta2)

Em seguida, implementamos o algoritmo de retropropagação para reduzir o erro no conjunto de treino, calculando a atualização dos parâmetros. Em primeiro lugar, precisamos de calcular o gradiente da função sigmoide que implementámos anteriormente.

In [17]:
def sigmoid_gradient(z):
    z = sigmoid(z)*(1 - sigmoid(z))
    return z

Implementamos agora a propagação para trás para calcular os gradientes.

In [18]:
def backprop(params, input_size, hidden_size, num_labels, X, y):
    # Lo primero que debemos hacer es una progagación hacia adelante y calcular el coste
    # Para ello, puedes copiar el código de la función implementada para calcular el coste (sin regularización)
    # desempaquetamos las matrices con los parámetros para cada capa
    theta1 = np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1)))
    theta2 = np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1)))
    
    # Ejecutamos las propagación hacia adelante para obtener las salidas para cada ejemplo
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # Calculamos el coste
    m = X.shape[0]
    J = 1/m * np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h))
    
    # A partir de aquí comienza la implementación de backpropagation
    
    # Inicializamos los acumuladores delta1  y delta2 a ceros, con las dismensiones de los theta1 y theta2
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    # Aunque podríamos vectorizarlo, vamos a hacerlo para cada ejemplo
    for t in range(m):
        # Obtenemos lo que necesitamos del ejemplo t (cálculos obtenidos en la propagación hacia adelante)
        # Para usar las fórmulas tal y como aparecen en teoría, vamos a coger todos los vectores en forma de columna (reshape(-1,1))
        a1t = a1[t,:].reshape(-1, 1) 
        z2t = z2[t,:].reshape(-1, 1) 
        a2t = a2[t,:].reshape(-1, 1)
        ht = h[t,:].reshape(-1, 1)  
        yt = y[t,:].reshape(-1, 1)  
        
        # Calculamos el error en la capa de salida (delta3), almacenar en d3t
        d3t = ht - yt 
        
        # Para calcular el error en la capa oculta (delta2) necesitamos añadir un uno al inicio del vector z2
        z2t = np.vstack((np.array([1]), z2t))
        
        # Calculamos d2 a partir del error de la capa de salida, los parámetros en theta2 y el gradiente de z2t
        d2t = np.dot(theta2.T, d3t) * sigmoid_gradient(z2t)
        
        # Ya podemos calcular los gradientes a partir de los errores
        # Para calcular el gradiente de los theta1, tenemos en cuenta el error en la capa oculta d2t
        # Ten en cuenta que habrá que no necesitamos el primer elemento de d2t.
        # Se debe acumular el gradiente en delta1 y delta2
        delta1 = delta1 + d2t[1:] * a1t.T
        delta2 = delta2 + d3t * a2t.T 
    
    # Calculamos el gradiente finalmente dividiendo entre el número de ejemplos
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # Para pasar los gradientes a minimize los ponemos en un vector
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad


In [19]:
J, grad = backprop(params, input_size, hidden_size, num_labels, Xtrain, y_onehot)
print('O custo obtido: ' + str(J))

O custo obtido: 2.805747308593745


Depois de criar estas funções e de as testar, podemos agora treinar a rede e utilizá-la para fazer previsões. Utilizamos o método  *minimize*.

In [20]:
# Minimizar la función objetivo que acabamos de definir
fmin = minimize(fun = backprop, x0 = params, args = (input_size, hidden_size, num_labels, Xtrain, y_onehot), 
                method = 'TNC', jac = True, options = {'maxiter': 250})
#fmin

C:\Users\ainho\AppData\Local\Temp\ipykernel_9224\3614032905.py:2: OptimizeWarning: Unknown solver options: maxiter
  fmin = minimize(fun = backprop, x0 = params, args = (input_size, hidden_size, num_labels, Xtrain, y_onehot),
C:\Users\ainho\AppData\Local\Temp\ipykernel_9224\3499779148.py:13: RuntimeWarning: divide by zero encountered in log
  J = 1/m * np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h))
C:\Users\ainho\AppData\Local\Temp\ipykernel_9224\3499779148.py:13: RuntimeWarning: invalid value encountered in multiply
  J = 1/m * np.sum(-y * np.log(h) - (1 - y) * np.log(1 - h))


In [21]:
# Desempaquetamos los parámetros obtenidos como resultado del entrenamiento almacenados en fmin.x
# Crea las variables theta1 y theta2
theta1 = np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1)))
theta2 = np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1)))
    
# Utilizamos los parámetros desempaquetados con la propagación hacia adelante para obtener la predicción para nuestros ejemplos
a1, z2, a2, z3, h = forward_propagate(Xval, theta1, theta2)

# Finalmente, para obtener la clase para cada ejemplo, buscamos de las diez salidas cuál es la más alta
# y usamos su índice como valor predicho (utilizar np.argmax con el el valor adecuado para el parámetro axis).
ypred = np.argmax(h, axis=1).reshape(-1,1)

Por fim, calculamos a precisão do modelo em treino para verificar o seu desempenho.

In [22]:
# Calcula el accuracy en los datos de entrenamiento
accval = accuracy_score(ypred, yval)
print("Precisão no conjunto de validação: {}".format(accval))

best_accs['Redes Neuronales'] = accval

Precisão no conjunto de validação: 0.8429752066115702


Por fim, testamos vários valores dos diferentes parâmetros utilizados pelo modelo para obter a melhor combinação, de modo a obter a maior precisão no conjunto de validação.

In [23]:
#Listas de parámetros a explorar
hidden_layer_sizes_options = [(50,25), (100,50), (100,100)]
alpha_options = [0.01, 0.1, 1.0]
max_iter_options = [500, 750, 1000]

best_params_combination = {}
best_accval = 0.0

for hidden_layer_sizes in hidden_layer_sizes_options:
        for alpha in alpha_options:
            for max_iter in max_iter_options:
                modelo = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes,
                                       activation = 'relu',
                                       alpha = alpha,
                                       max_iter = max_iter,
                                       random_state = 42
                                      )
                modelo.fit(Xtrain, ytrain.ravel())
                ypred = modelo.predict(Xval)
                accval = accuracy_score(ypred, yval)
                if accval > best_accval:
                    best_accval = accval
                    best_hidden_layer = hidden_layer_sizes,
                    best_alpha = alpha,
                    best_max_iter = max_iter
                best_params_combination ['best_hidden_layer'] = best_hidden_layer  
                best_params_combination ['best_alpha'] = best_alpha  
                best_params_combination ['best_max_iter'] = best_max_iter 
                
print('Os melhores hiperparâmetros são: {} iterações, {} neurónios em cada camada oculta e {} como o valor de alfa.' .format(best_max_iter, best_hidden_layer, best_alpha))
print('A precisão dos dados de validação é de: {}'.format(best_accval))

best_accs['Red Neuronal Multicapa'] = best_accval
best_params['Red Neuronal Multicapa'] = best_params_combination

C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\neural_network\_multi

Os melhores hiperparâmetros são: 500 iterações, ((100, 100),) neurónios em cada camada oculta e (0.1,) como o valor de alfa.
A precisão dos dados de validação é de: 0.9752066115702479


Concluímos a partir destes resultados que, por enquanto, o modelo de rede neural é o melhor, uma vez que obtemos uma melhor precisão do que com os modelos anteriores. Mesmo assim, vamos tentar outro.

In [24]:
best_params_combination

{'best_hidden_layer': ((100, 100),),
 'best_alpha': (0.1,),
 'best_max_iter': 500}

#### SVMs

Agora, vamos utilizar os algoritmos SVM para treinar o conjunto de dados e prever a precisão. Para tal, vamos testar com várias combinações de parâmetros qual delas tem a melhor precisão de validação. Aprendemos, portanto, várias SVMs com todas as combinações de valores C e gama e escolhemos a que tem o menor erro de validação. Realizamos este processo de afinação de hiper-parâmetros diretamente com a função *GridSearchCV()*.

In [25]:
C_values = [0.01, 0.1, 1, 10]
gamma_values = [0.01, 0.1, 1, 10]
kernel= ['rbf', 'linear']

# Creamos el diccionario de parámetros a optimizar con los valores de C y gamma de las listas
param_grid = {'kernel': kernel, 'C': C_values, 'gamma': gamma_values}

# Haz la llamada a la función GridSearchCV con un clasificador SVC, el diccionario de parámetros definido y 3 particiones
clasificador = SVC()
clasificadores = GridSearchCV(clasificador, param_grid, scoring = 'accuracy', cv = 3)

# Entrenamos los modelos 
clasificadores.fit(Xtrain, ytrain)

# Creamos un modelo SVC con los mejores parámetros obenidos
svc_best_params = SVC(kernel = clasificadores.best_params_['kernel'], gamma = clasificadores.best_params_['gamma'], C = clasificadores.best_params_['C'])

# Entrenamos la SVM con los datos de train
svc_best_params.fit(Xtrain, ytrain)

# Obtenemos el accuracy en train y validación
acctrain = svc_best_params.score(Xtrain, ytrain)
accval = svc_best_params.score(Xval, yval)

print("Precisão no conjunto de treino: {}".format(acctrain))
print("Precisão no conjunto de validação: {}".format(accval))

best_accs['SVMs'] = accval
best_params['SVMs'] = clasificadores.best_params_

Precisão no conjunto de treino: 0.9810874704491725
Precisão no conjunto de validação: 0.9393939393939394


### ÁRVORES DE DECISÃO

In [26]:
# ARBOL DE DECISION
arbol=DecisionTreeClassifier(random_state=123)

criterio=['gini','entropy']
min_samples_split=[2,5,10]
min_samples_leaf=[1,2,4]
ccp_alpha=[0.0001, 0.001,0.002,0.003]

param_grid = {'criterion': criterio, 'min_samples_split': min_samples_split, 'min_samples_leaf':min_samples_leaf, 'ccp_alpha':ccp_alpha}

arbol = GridSearchCV(DecisionTreeClassifier(), param_grid)
arbol.fit(Xtrain,ytrain.ravel())
mejores_parametros=arbol.best_params_
print("Parâmetros selecionados: ", mejores_parametros)


Parâmetros selecionados:  {'ccp_alpha': 0.0001, 'criterion': 'entropy', 'min_samples_leaf': 1, 'min_samples_split': 2}


In [27]:
acc_train=arbol.score(Xtrain,ytrain)
acc_val=arbol.score(Xval,yval)
print("Precisão no conjunto de treino: {}".format(acc_train))
print("Precisão no conjunto de validação: {}".format(acc_val))

best_accs['ArbolDecision'] = accval
best_params['ArbolDecision'] = clasificadores.best_params_

Precisão no conjunto de treino: 1.0
Precisão no conjunto de validação: 0.977961432506887


### Random Forest

In [28]:
criterio=['gini','entropy']
max_features =['auto', 'sqrt', 'log2']
min_samples_split=[2,5,10]
n_estimators = [50, 100, 200, 300]

param_grid = {'criterion': criterio, 'max_features': max_features, 'min_samples_split': min_samples_split, 'n_estimators':n_estimators}

arbol = GridSearchCV(RandomForestClassifier(), param_grid)
arbol.fit(Xtrain,ytrain.ravel())
mejores_parametros=arbol.best_params_
print("Parâmetros selecionados: ",mejores_parametros)


C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
C:\Users\ainho\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

Parâmetros selecionados:  {'criterion': 'entropy', 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 300}


In [29]:
RandomForest = RandomForestClassifier(criterion = mejores_parametros['criterion'], max_features =mejores_parametros['max_features'], min_samples_split=mejores_parametros['min_samples_split'], n_estimators=mejores_parametros['n_estimators'])
rf = RandomForest.fit(Xtrain, ytrain)

pred_train = rf.predict(Xtrain)

pred_val = rf.predict(Xval)

acc_train=rf.score(Xtrain,ytrain)
print("Precisão no conjunto de treino: ",acc_train)

acc_val=rf.score(Xval,yval)
print("Precisão no conjunto de validação: ",acc_val)


best_accs['Random Forest'] = accval
best_params['Random Forest'] = clasificadores.best_params_

Precisão no conjunto de treino:  1.0
Precisão no conjunto de validação:  0.9338842975206612


### Modelo escolhido

Escolheremos o melhor modelo utilizando os dicionários em que armazenámos os diferentes classificadores e as respectivas precisões na validação. 

In [30]:
best_accs

{'Regresión Logística': 0.6639118457300276,
 'Regresión Logística con Regularización': 0.9421487603305785,
 'KNN': 0.8815426997245179,
 'KNN distancias': 0.8815426997245179,
 'Naive Bayes': 0.581267217630854,
 'Redes Neuronales': 0.8429752066115702,
 'Red Neuronal Multicapa': 0.9752066115702479,
 'SVMs': 0.9393939393939394,
 'ArbolDecision': 0.9393939393939394,
 'Random Forest': 0.9393939393939394}

In [31]:
# Obtener una lista con todos los valores
lista_de_valores = list(best_accs.values())

#Obtenemos el mejor accuracy de entre todos los que hemos ido obteniendp
mejorAcc = np.max(lista_de_valores)

#Obtenemos cuál es el clasificador que proporciona ese accuracy
mejorClasif = None
for clave, valor in best_accs.items():
    if valor == mejorAcc:
        mejorClasif = clave
        break

print('\nO melhor classificador é  {}, com precisão no conjunto de validação {}'.format(mejorClasif, mejorAcc))


O melhor classificador é  Red Neuronal Multicapa, com precisão no conjunto de validação 0.9752066115702479


Vamos agora utilizar o melhor modelo (árvore de decisão como modelo de seleção de variáveis) para prever os dados do nosso conjunto de teste e obter a precisão correspondente. Para o fazer, começamos por ler os dados de teste. 

In [32]:
mejores_parametros = best_params['Red Neuronal Multicapa']
mejores_parametros

{'best_hidden_layer': ((100, 100),),
 'best_alpha': (0.1,),
 'best_max_iter': 500}

In [33]:
from sklearn.feature_selection import SelectKBest, f_classif

# Selección de características utilizando SelectKBest
selector = SelectKBest(score_func=f_classif, k='all')  # Cambia 'all' a un número específico si lo deseas
X_train_selected = selector.fit_transform(Xtrain, ytrain)

# Ahora puedes entrenar tu modelo con las características seleccionadas
modeloRNM = MLPClassifier(
    hidden_layer_sizes=(100,100),
    activation='relu',
    alpha=0.1,
    max_iter=500,
    random_state=42
)

RNM = modeloRNM.fit(X_train_selected, ytrain.ravel())

# Para transformar el conjunto de prueba
X_test_selected = selector.transform(Xtest)
ypredTest = RNM.predict(X_test_selected)

In [34]:
accuracy_score(ypredTest, ytest)

0.9710982658959537

### Conclusões

Neste processo de seleção de modelos, avaliámos diferentes técnicas de aprendizagem automática, cada uma com as suas próprias caraterísticas. Após análise, o modelo *Red Neuronal Multicapa* destacou-se pelo seu desempenho e exatidão, com uma exatidão de 97% na validação. 